In [77]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from wordcloud import WordCloud

import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# import string library for text preprocessing
import string

from nltk.stem import WordNetLemmatizer
import ast

In [40]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [41]:
df = pd.read_csv('../data/merged/imdb_movies_min_votes_500.csv')
print(df.shape)
df.head()

(7553, 16)


,title,year,description,image_link,imdb_id,rating_count,best_rating,worst_rating,rating,genre,keywords,trailer_link,actors,director,synopsis,summary
0,Gladiator,2000,A former Roman General sets out to exact venge...,https://m.media-amazon.com/images/M/MV5BMDliMm...,tt0172495,1617220,10,1,8.5,"['Action', 'Adventure', 'Drama']","['roman empire', 'gladiator', 'slavery', 'comb...",https://www.imdb.com/video/vi2628367897/,"['Russell Crowe', 'Joaquin Phoenix', 'Connie N...",['Ridley Scott'],"Shouting ""Roma Invicta!"" as his forces attack,...",A former Roman General sets out to exact venge...
1,Unbreakable,2000,A man learns something extraordinary about him...,https://m.media-amazon.com/images/M/MV5BMDIwMj...,tt0217869,440860,10,1,7.3,"['Drama', 'Mystery', 'Sci-Fi']","['supernatural power', 'train crash', 'superhe...",https://www.imdb.com/video/vi1103346201/,"['Bruce Willis', 'Samuel L. Jackson', 'Robin W...",['M. Night Shyamalan'],The film opens as we see a baby being born in ...,A man learns something extraordinary about him...
2,Snatch,2000,"Unscrupulous boxing promoters, violent bookmak...",https://m.media-amazon.com/images/M/MV5BMTA2ND...,tt0208092,909299,10,1,8.2,"['Comedy', 'Crime']","['diamond', 'boxer', 'narrated by character', ...",https://www.imdb.com/video/vi1558577433/,"['Jason Statham', 'Brad Pitt', 'Stephen Graham']",['Guy Ritchie'],After stealing an 86-carat (17 g) diamond in a...,"Unscrupulous boxing promoters, violent bookmak..."
3,AmericanPsycho,2000,A wealthy New York City investment banking exe...,https://m.media-amazon.com/images/M/MV5BZTM2ZG...,tt0144084,713635,10,1,7.6,"['Crime', 'Drama', 'Horror']","['narcissism', 'materialism', 'serial murder',...",https://www.imdb.com/video/vi4060743449/,"['Christian Bale', 'Justin Theroux', 'Josh Luc...",['Mary Harron'],A white background. Red drops begin to fall pa...,A wealthy New York City investment banking exe...
4,RequiemforaDream,2000,The drug-induced utopias of four Coney Island ...,https://m.media-amazon.com/images/M/MV5BOTdiNz...,tt0180093,897414,10,1,8.3,['Drama'],"['drug addiction', 'sex show', 'sex scene', 'p...",https://www.imdb.com/video/vi2156069145/,"['Ellen Burstyn', 'Jared Leto', 'Jennifer Conn...",['Darren Aronofsky'],Harry Goldfarb (Jared Leto) and Tyrone Love (M...,The drug-induced utopias of four Coney Island ...


In [42]:
df.columns

Index(['title', 'year', 'description', 'image_link', 'imdb_id', 'rating_count',
       'best_rating', 'worst_rating', 'rating', 'genre', 'keywords',
       'trailer_link', 'actors', 'director', 'synopsis', 'summary'],
      dtype='object')

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7553 entries, 0 to 7552
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         7553 non-null   object 
 1   year          7553 non-null   int64  
 2   description   7553 non-null   object 
 3   image_link    7553 non-null   object 
 4   imdb_id       7553 non-null   object 
 5   rating_count  7553 non-null   int64  
 6   best_rating   7553 non-null   int64  
 7   worst_rating  7553 non-null   int64  
 8   rating        7553 non-null   float64
 9   genre         7552 non-null   object 
 10  keywords      7553 non-null   object 
 11  trailer_link  6778 non-null   object 
 12  actors        7553 non-null   object 
 13  director      7553 non-null   object 
 14  synopsis      3432 non-null   object 
 15  summary       7551 non-null   object 
dtypes: float64(1), int64(4), object(11)
memory usage: 944.3+ KB


# Missing Values Handling

In [44]:
df.isnull().sum()

title              0
year               0
description        0
image_link         0
imdb_id            0
rating_count       0
best_rating        0
worst_rating       0
rating             0
genre              1
keywords           0
trailer_link     775
actors             0
director           0
synopsis        4121
summary            2
dtype: int64

In [45]:
df[df['summary'].isnull()][['description', 'synopsis']]


,description,synopsis
169,Islam: Empire of Faith is a documentary series...,Islam: Empire of Faith is a documentary series...
4215,"On Aug. 9, 1974, Richard Milhous Nixon became ...","On Aug. 9, 1974, Richard Milhous Nixon became ..."


In [46]:
# replace missing values in synopsis with summary and if summary is empty replace with description
df['synopsis'].fillna(df['summary'], inplace=True)
df['synopsis'].fillna(df['description'], inplace=True)

In [47]:
df['synopsis'].isnull().sum()

0

In [48]:
# drop description and summary columns
df.drop(columns=['description', 'summary'], inplace=True)

In [49]:
df.columns

Index(['title', 'year', 'image_link', 'imdb_id', 'rating_count', 'best_rating',
       'worst_rating', 'rating', 'genre', 'keywords', 'trailer_link', 'actors',
       'director', 'synopsis'],
      dtype='object')

In [50]:
df.isnull().sum()

title             0
year              0
image_link        0
imdb_id           0
rating_count      0
best_rating       0
worst_rating      0
rating            0
genre             1
keywords          0
trailer_link    775
actors            0
director          0
synopsis          0
dtype: int64

In [51]:
df[df['genre'].isnull()]

,title,year,image_link,imdb_id,rating_count,best_rating,worst_rating,rating,genre,keywords,trailer_link,actors,director,synopsis
6882,BeguiledCompany,2021,https://m.media-amazon.com/images/M/MV5BZTFlZT...,tt7884922,872,10,1,6.7,NaN,"['group', 'two word title']",NaN,"['Chance Gilliam', 'Jess Tomasko', 'Devyn Will...",['Tristan James Jensen'],A group of zillennial friends come together an...


In [52]:
df.loc[df['imdb_id'] == 'tt7884922', 'genre'] = "['Action', 'Thriller', 'Crime Fiction']"

In [53]:
df.isnull().sum()

title             0
year              0
image_link        0
imdb_id           0
rating_count      0
best_rating       0
worst_rating      0
rating            0
genre             0
keywords          0
trailer_link    775
actors            0
director          0
synopsis          0
dtype: int64

In [54]:
df[df['trailer_link'].isnull()].head()

,title,year,image_link,imdb_id,rating_count,best_rating,worst_rating,rating,genre,keywords,trailer_link,actors,director,synopsis
38,Dinosaur,2000,https://m.media-amazon.com/images/M/MV5BZWQ1NT...,tt0130623,67113,10,1,6.4,"['Animation', 'Adventure', 'Drama']","['dinosaur', 'carnotaurus', 'lemur', 'egg', 'm...",NaN,"['D.B. Sweeney', 'Julianna Margulies', 'Samuel...","['Eric Leighton', 'Ralph Zondag']","In a dinosaur breeding ground, a Carnotaurus t..."
54,U-,2000,https://m.media-amazon.com/images/M/MV5BMDE0Yz...,tt0141926,87314,10,1,6.6,"['Action', 'War']","['submarine movie', 'world war two', 'military...",NaN,"['Matthew McConaughey', 'Bill Paxton', 'Harvey...",['Jonathan Mostow'],German U-boat U-571 is immobilized by a Britis...
62,Shaft,2000,https://m.media-amazon.com/images/M/MV5BYmJhNW...,tt0162650,80738,10,1,6.0,"['Action', 'Crime', 'Thriller']","['fourth part', 'uncle nephew relationship', '...",NaN,"['Samuel L. Jackson', 'Vanessa Williams', 'Chr...",['John Singleton'],"In 1998, NYPD Detective John Shaft II (Samuel ..."
89,WakingtheDead,2000,https://m.media-amazon.com/images/M/MV5BNDY1OD...,tt0127349,7596,10,1,6.4,"['Drama', 'Mystery', 'Romance']","['female nudity', 'based on novel', 'father da...",NaN,"['Billy Crudup', 'Bill Haugland', 'Nelson Land...",['Keith Gordon'],The first scene of the movie introduces Fieldi...
97,Maze,2000,https://m.media-amazon.com/images/M/MV5BMTkwNz...,tt0246072,922,10,1,6.3,"['Drama', 'Romance']","['female star appears nude', 'nude model', 'fe...",NaN,"['Rob Morrow', 'Laura Linney', 'Craig Sheffer']",['Rob Morrow'],An artist with Tourette Syndrome falls in love...


# Feature Selection

In [55]:
df.columns

Index(['title', 'year', 'image_link', 'imdb_id', 'rating_count', 'best_rating',
       'worst_rating', 'rating', 'genre', 'keywords', 'trailer_link', 'actors',
       'director', 'synopsis'],
      dtype='object')

In [56]:
df['best_rating'].value_counts()

best_rating
10    7553
Name: count, dtype: int64

In [57]:
df['worst_rating'].value_counts()

worst_rating
1    7553
Name: count, dtype: int64

In [58]:
# drop worst_rating and best_rating columns as they don't have significant meaning
df.drop(columns=['worst_rating', 'best_rating'], inplace=True)

In [59]:
df.columns

Index(['title', 'year', 'image_link', 'imdb_id', 'rating_count', 'rating',
       'genre', 'keywords', 'trailer_link', 'actors', 'director', 'synopsis'],
      dtype='object')

# Duplicates Check

In [60]:
df.duplicated().sum()

0

# Clean synopsis

In [61]:
# Convert text to lowercase
def convert_to_lower_case(text):
  return text.lower()

In [62]:
def remove_punctuation(text):
  # Create a translation table to remove punctuation
  translator = str.maketrans('', '', string.punctuation)

  # Remove punctuation using the translation table
  text_without_punct = text.translate(translator)

  return text_without_punct

In [63]:
def tokenize(text):
  # Tokenization
  return word_tokenize(text)

In [64]:
# Remove stopwords
def remove_stop_words(tokens):
  stop_words = set(stopwords.words('english'))
  return [word for word in tokens if word not in stop_words]

In [65]:
def lemmatize(tokens):
  # Lemmatization
  lemmatizer = WordNetLemmatizer()
  return [lemmatizer.lemmatize(word) if lemmatizer.lemmatize(word) is not None else word for word in tokens]

In [66]:
# Remove special characters and numbers
def remove_special_chars(tokens):
    return [re.sub('[^A-Za-z]+', '', word) for word in tokens]

In [67]:
# function to do preprocessing text
def preprocess_text(text):
  lower_text = convert_to_lower_case(text) # converting to lowercase
  removed_punctuation = remove_punctuation(lower_text) # removing punctuation
  tokens = tokenize(removed_punctuation) # tokenize words
  tokens = remove_stop_words(tokens) # removing stop words
  tokens = lemmatize(tokens) # lemmatize tokens
  tokens = remove_special_chars(tokens) # remove special characters from token

  preprocessed_text = ' '.join(tokens) # final preprocessed text
  return preprocessed_text

In [68]:
# Apply function on text column
df['processed_plot'] = df['synopsis'].apply(preprocess_text)

In [69]:
df['processed_plot'].isnull().sum()

0

In [71]:
df.iloc[0]['processed_plot']

'shouting rom invicta force attack general maximus decimus meridius russell crowe lead roman army victory germanic barbarian year  ad ending prolonged war earning esteem elderly emperor marcus aurelius although dying aurelius son commodus joaquin phoenix decides appoint temporary leadership morallyupstanding maximus desire eventually return power roman senate aurelius informs maximus offer time consider informing commodus bout jealousy murder fatherdeclaring emperor commodus asks maximus loyalty maximus realizing commodus involvement emperor death refuse commodus order maximus execution dispatch praetorian guard murder maximus wife son maximus narrowly escape execution race home discover family charred crucified body smoldering ruin villa burying wife son grieving maximus succumbs exhaustion collapse gravesslave trader find maximus take zucchabar rugged province north africa purchased proximo head local gladiator school distraught nihilistic death family betrayal empire maximus initial

In [72]:
# drop synopsis column
df.drop(columns=['synopsis'], inplace=True)

In [73]:
df.columns

Index(['title', 'year', 'image_link', 'imdb_id', 'rating_count', 'rating',
       'genre', 'keywords', 'trailer_link', 'actors', 'director',
       'processed_plot'],
      dtype='object')

# Clean features with multiple values

In [74]:
df.columns

Index(['title', 'year', 'image_link', 'imdb_id', 'rating_count', 'rating',
       'genre', 'keywords', 'trailer_link', 'actors', 'director',
       'processed_plot'],
      dtype='object')

In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7553 entries, 0 to 7552
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           7553 non-null   object 
 1   year            7553 non-null   int64  
 2   image_link      7553 non-null   object 
 3   imdb_id         7553 non-null   object 
 4   rating_count    7553 non-null   int64  
 5   rating          7553 non-null   float64
 6   genre           7553 non-null   object 
 7   keywords        7553 non-null   object 
 8   trailer_link    6778 non-null   object 
 9   actors          7553 non-null   object 
 10  director        7553 non-null   object 
 11  processed_plot  7553 non-null   object 
dtypes: float64(1), int64(2), object(9)
memory usage: 708.2+ KB


In [76]:
list_cols = ['keywords', 'actors', 'director', 'genre']
df[list_cols].isnull().sum()

keywords    0
actors      0
director    0
genre       0
dtype: int64

In [78]:
list_cols = ['keywords', 'actors', 'director', 'genre']
for col in list_cols:
    df[col] = df[col].apply(ast.literal_eval)

In [79]:
df.head()

,title,year,image_link,imdb_id,rating_count,rating,genre,keywords,trailer_link,actors,director,processed_plot
0,Gladiator,2000,https://m.media-amazon.com/images/M/MV5BMDliMm...,tt0172495,1617220,8.5,"[Action, Adventure, Drama]","[roman empire, gladiator, slavery, combat, bat...",https://www.imdb.com/video/vi2628367897/,"[Russell Crowe, Joaquin Phoenix, Connie Nielsen]",[Ridley Scott],shouting rom invicta force attack general maxi...
1,Unbreakable,2000,https://m.media-amazon.com/images/M/MV5BMDIwMj...,tt0217869,440860,7.3,"[Drama, Mystery, Sci-Fi]","[supernatural power, train crash, superhero, c...",https://www.imdb.com/video/vi1103346201/,"[Bruce Willis, Samuel L. Jackson, Robin Wright]",[M. Night Shyamalan],film open see baby born west philadelphia elij...
2,Snatch,2000,https://m.media-amazon.com/images/M/MV5BMTA2ND...,tt0208092,909299,8.2,"[Comedy, Crime]","[diamond, boxer, narrated by character, cockne...",https://www.imdb.com/video/vi1558577433/,"[Jason Statham, Brad Pitt, Stephen Graham]",[Guy Ritchie],stealing carat g diamond heist antwerp franki...
3,AmericanPsycho,2000,https://m.media-amazon.com/images/M/MV5BZTM2ZG...,tt0144084,713635,7.6,"[Crime, Drama, Horror]","[narcissism, materialism, serial murder, socio...",https://www.imdb.com/video/vi4060743449/,"[Christian Bale, Justin Theroux, Josh Lucas]",[Mary Harron],white background red drop begin fall past open...
4,RequiemforaDream,2000,https://m.media-amazon.com/images/M/MV5BOTdiNz...,tt0180093,897414,8.3,[Drama],"[drug addiction, sex show, sex scene, prostitu...",https://www.imdb.com/video/vi2156069145/,"[Ellen Burstyn, Jared Leto, Jennifer Connelly]",[Darren Aronofsky],harry goldfarb jared leto tyrone love marlon w...


In [80]:
# remove unnecessary columns
unrelevant_cols = ['image_link', 'trailer_link']
df.drop(columns=unrelevant_cols, inplace=True)

In [81]:
df.head()

,title,year,imdb_id,rating_count,rating,genre,keywords,actors,director,processed_plot
0,Gladiator,2000,tt0172495,1617220,8.5,"[Action, Adventure, Drama]","[roman empire, gladiator, slavery, combat, bat...","[Russell Crowe, Joaquin Phoenix, Connie Nielsen]",[Ridley Scott],shouting rom invicta force attack general maxi...
1,Unbreakable,2000,tt0217869,440860,7.3,"[Drama, Mystery, Sci-Fi]","[supernatural power, train crash, superhero, c...","[Bruce Willis, Samuel L. Jackson, Robin Wright]",[M. Night Shyamalan],film open see baby born west philadelphia elij...
2,Snatch,2000,tt0208092,909299,8.2,"[Comedy, Crime]","[diamond, boxer, narrated by character, cockne...","[Jason Statham, Brad Pitt, Stephen Graham]",[Guy Ritchie],stealing carat g diamond heist antwerp franki...
3,AmericanPsycho,2000,tt0144084,713635,7.6,"[Crime, Drama, Horror]","[narcissism, materialism, serial murder, socio...","[Christian Bale, Justin Theroux, Josh Lucas]",[Mary Harron],white background red drop begin fall past open...
4,RequiemforaDream,2000,tt0180093,897414,8.3,[Drama],"[drug addiction, sex show, sex scene, prostitu...","[Ellen Burstyn, Jared Leto, Jennifer Connelly]",[Darren Aronofsky],harry goldfarb jared leto tyrone love marlon w...


In [83]:
df.reset_index(drop=True, inplace=True)

In [84]:
df.head()

,title,year,imdb_id,rating_count,rating,genre,keywords,actors,director,processed_plot
0,Gladiator,2000,tt0172495,1617220,8.5,"[Action, Adventure, Drama]","[roman empire, gladiator, slavery, combat, bat...","[Russell Crowe, Joaquin Phoenix, Connie Nielsen]",[Ridley Scott],shouting rom invicta force attack general maxi...
1,Unbreakable,2000,tt0217869,440860,7.3,"[Drama, Mystery, Sci-Fi]","[supernatural power, train crash, superhero, c...","[Bruce Willis, Samuel L. Jackson, Robin Wright]",[M. Night Shyamalan],film open see baby born west philadelphia elij...
2,Snatch,2000,tt0208092,909299,8.2,"[Comedy, Crime]","[diamond, boxer, narrated by character, cockne...","[Jason Statham, Brad Pitt, Stephen Graham]",[Guy Ritchie],stealing carat g diamond heist antwerp franki...
3,AmericanPsycho,2000,tt0144084,713635,7.6,"[Crime, Drama, Horror]","[narcissism, materialism, serial murder, socio...","[Christian Bale, Justin Theroux, Josh Lucas]",[Mary Harron],white background red drop begin fall past open...
4,RequiemforaDream,2000,tt0180093,897414,8.3,[Drama],"[drug addiction, sex show, sex scene, prostitu...","[Ellen Burstyn, Jared Leto, Jennifer Connelly]",[Darren Aronofsky],harry goldfarb jared leto tyrone love marlon w...


In [85]:
df.to_csv('../data/processed/preprocessed_dataset.csv', index=False)